In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image
import random

from keras import models, layers, optimizers, callbacks

from keras.preprocessing.image import ImageDataGenerator

In [ ]:
DATASET_DIR = '/kaggle/input/gtsrb-german-traffic-sign'

TRAIN_DIR = DATASET_DIR + '/Train'
TEST_DIR = DATASET_DIR + '/Test'
MAX_EPOCHS = 100
BATCH_SIZE = 128
LR = 1e-3

In [ ]:
train_x = []
train_y = []

for i in range(43):
    print(f'Reading class: {i}')
    CLASS_DIR = f'{TRAIN_DIR}/{i}/'
    
    label = np.zeros((43, ))
    label[i] = 1
    
    for file in random.choices(os.listdir(CLASS_DIR), k=1000):
        
        image = Image.open(CLASS_DIR + file)
        image = image.resize((64, 64))
        image = np.asarray(image)
        
        train_x.append(image)
        train_y.append(label)
        
@tf.function
def image_augment(x, y): 
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
        
    if p_rotate > .8:
        x = tf.image.rot90(x, k=3) 
    elif p_rotate > .6:
        x = tf.image.rot90(x, k=2) 
    elif p_rotate > .4:
        x = tf.image.rot90(x, k=1)
        
    x = tf.image.random_flip_left_right(x)
    x = tf.image.random_flip_up_down(x)
    
    if p_spatial > .75:
        x = tf.image.transpose(x)
        
    return x, y
    
train_x = np.asarray(train_x, dtype='float32') / 255
train_y = np.asarray(train_y, dtype='float32')

train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))

train_dataset = train_dataset.map(image_augment)

train_dataset = train_dataset.shuffle(2048)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
test_x = []
test_y = []
test_df = pd.read_csv(DATASET_DIR + '/Test.csv')

for file in random.choices(os.listdir(TEST_DIR), k=1000):
    if not '.png' in file:
        continue
    
    class_id = test_df[test_df['Path'] == ('Test/' + file)]['ClassId']
    
    image = Image.open(TEST_DIR + '/' + file)
    image = image.resize((64, 64))
    image = np.asarray(image)
    
    label = np.zeros(43)
    label[class_id] = 1

    test_x.append(image)
    test_y.append(label)

    
test_x = np.asarray(test_x, dtype='float32') / 255
test_y = np.asarray(test_y, dtype='float32')


test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))

test_dataset = test_dataset.map(image_augment)

test_dataset = test_dataset.shuffle(2048)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), input_shape=(64, 64, 3), activation='relu'),
    layers.Conv2D(32, kernel_size=(3, 3), input_shape=(64, 64, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.BatchNormalization(),  
    
    layers.Flatten(),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Dense(43, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
model.summary()

In [ ]:
history = model.fit(train_dataset, validation_data=(test_dataset), epochs=MAX_EPOCHS, callbacks=[
    callbacks.EarlyStopping('val_accuracy', patience=5)
])

In [ ]:
model.save('model.h5')

In [ ]:
history = history.history
plt.figure()
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.grid()
plt.legend(['loss', 'val_loss'])
plt.show()

plt.figure()
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.grid()
plt.legend(['accuracy', 'val_accuracy'])
plt.show()